In [1]:
import os

os.getcwd()
os.chdir("../../")
os.getcwd()

'/data01/dl23vitcas/dl_project'

In [2]:
import sys
import argparse
import os

sys.argv = ["view", "--config", "config/single_task_object_detection.yaml"]

parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
args = parser.parse_args()

print(args.config)

config/single_task_object_detection.yaml


In [3]:
from dataloader import VOC08Attr, extract_positive_and_negative
from torchvision.transforms import transforms
from config_experiments import config
import torch

In [4]:
transform_train = transforms.Compose(
    [
        transforms.Resize(size=600, max_size=1000),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=config["transform"]["mean"], std=config["transform"]["std"]
        ),
    ]
)

In [5]:
train_data = VOC08Attr(train=True, transform=transform_train)

In [32]:
list_img_size = []
for i, (image, img_size, gt_class, gt_bbox, gt_attributes, ss_rois) in enumerate(
    train_data
):
    list_img_size.append([img_size[0], img_size[1]])

print(list_img_size)
t_size = torch.tensor(list_img_size, dtype=torch.float)
print(t_size.shape)
mean = torch.mean(t_size, dim=0)
print(mean)

[[500, 442], [500, 327], [480, 272], [333, 500], [500, 375], [500, 333], [500, 375], [500, 444], [500, 375], [500, 375], [500, 375], [448, 336], [483, 500], [500, 375], [500, 375], [500, 375], [376, 500], [500, 375], [375, 500], [500, 375], [333, 500], [500, 357], [500, 375], [500, 375], [500, 375], [500, 375], [500, 375], [500, 308], [500, 375], [500, 333], [500, 333], [500, 332], [500, 332], [500, 375], [500, 375], [500, 331], [494, 500], [500, 375], [486, 348], [500, 333], [500, 333], [500, 375], [500, 333], [500, 375], [500, 346], [500, 333], [500, 375], [500, 333], [500, 375], [500, 333], [334, 500], [500, 375], [500, 375], [500, 375], [500, 375], [500, 375], [500, 375], [500, 375], [500, 375], [500, 338], [500, 375], [500, 331], [500, 375], [500, 333], [500, 375], [500, 375], [500, 333], [500, 375], [500, 340], [375, 500], [500, 371], [500, 333], [500, 335], [500, 373], [500, 333], [500, 375], [500, 375], [500, 375], [403, 500], [500, 375], [500, 292], [500, 375], [500, 375], [33

In [10]:
all_offset = []
for i, (image, img_size, gt_class, gt_bbox, gt_attributes, ss_rois) in enumerate(
    train_data
):
    train_roi, train_cls, train_offset, train_attr = extract_positive_and_negative(
        gt_class, gt_bbox, gt_attributes, ss_rois
    )
    non_zero_indices = train_cls != 0
    filtered_train_offset = train_offset[non_zero_indices]
    if filtered_train_offset.size(0) > 0:
        all_offset.append(filtered_train_offset)
    if all_offset:
        tensor_offsets = torch.cat(all_offset)

In [11]:
"""
all_offset = []
for i, (image, img_size, gt_class, gt_bbox, gt_attributes, ss_rois) in enumerate(
    train_data
):
    train_roi, train_cls, train_offset, train_attr = extract_positive_and_negative(
        gt_class, gt_bbox, gt_attributes, ss_rois
    )
    all_offset.append(train_offset)
    tensor_offsets = torch.cat(all_offset)
"""

pass

In [12]:
mean = torch.mean(tensor_offsets, dim=0)
std = torch.std(tensor_offsets, dim=0)

In [13]:
mean

tensor([-0.0043, -0.0178, -0.4593, -0.7323])

In [14]:
std

tensor([0.2425, 0.2220, 0.3748, 0.4040])